In [ ]:
from sage.all import *

$$\left\{\begin{array}{rcl} s &\gets& \mathsf{Zr} \\ \beta & \gets& g(s)\end{array}\right.$$

In [ ]:
G=None

def KeyGen(lmbda):
    global G
    q = next_prime(2^lmbda)
    F = GF(q)
    G = F.multiplicative_generator()

    s = randint(1, q-1)
    beta = G^s

    return (s, beta)

$$key\;\gets\;(\,\vartheta \;d \gets \mathsf{ID}(\mathit{id})\;\centerdot\; s* d\,)$$

In [ ]:
def KeyExtract(id, s):
    d = ascii_integer(id)
    key = s * d
    return key



$$\mathsf{in\_ encrypt}(\mathit{id},x)\;\equiv$$ 
$$\vartheta\,d\gets\mathsf{ID}(\mathit{id})\,\centerdot\,\vartheta\,v\gets\mathsf{Zr}\,\centerdot\,\vartheta\,a\gets H(v\oplus x)\,\centerdot\,\vartheta\,\mu\gets\mathbf{ex}(\beta,d,a)\,\centerdot\,\langle x,v,a,\mu\rangle$$

In [ ]:
def in_encrypt(id, x, beta):
    d = ascii_integer(id)
    v = random_zmod_element(beta.order())
    a = hash(v + ascii_integer(x))
    mu = beta^d * G^a
    return (x, v, a, mu)

$$\mathsf{out\_ encrypt}(x,v,a,\mu)\;\equiv$$
$$\vartheta\,\alpha\gets g(a)\,\centerdot\,\vartheta\,v'\gets v\oplus f(\mu)\,\centerdot\,x'\gets x\oplus H(v)\,\centerdot\,\langle \alpha,v',x'\rangle$$

In [ ]:
def out_encrypt(x, v, a, mu):
    alpha = G^a
    v_prime = v ^ mu
    x_prime = x ^ hash(v)
    return (alpha, v_prime, x_prime)



$$\mathsf{Encrypt}(\mathit{id}, x) \;\equiv\;$$  
$$\vartheta \,x,v,a,\mu \gets \mathsf{in}(\mathit{id},x)\,\centerdot\,\mathsf{out}(x,v,a,\mu)$$

In [ ]:
def Encrypt(id, x, beta):
    x, v, a, mu = in_encrypt(id, x, beta)
    alpha, v_prime, x_prime = out_encrypt(x, v, a, mu)
    return (alpha, v_prime, x_prime)


$$\mathsf{in\_ decrypt}(\mathit{key},\alpha, v', x')\;\equiv$$
$$\vartheta\,\mu \gets \mathbf{ex}(\alpha,\mathit{key},1)\,\centerdot\,\vartheta\,v \gets v'\oplus f(\mu)\,\centerdot\,x\gets x'\oplus H(v)\,\centerdot\,\langle \alpha,v,x\rangle$$

In [ ]:
def in_decrypt(key, alpha, v_prime, x_prime):
    mu = alpha^key
    v = v_prime ^ mu
    x = x_prime ^ hash(v)
    return (alpha, v, x)



$$\mathsf{out\_ decrypt}(\alpha,v,x)\;\equiv$$
$$\vartheta\,a\gets H(v\oplus x)\,\centerdot\,\mathsf{if}\;\;\alpha\neq g(a)\;\;\mathsf{then}\;\;\texttt{fails}\;\;\mathsf{else}\;\;x$$

In [ ]:
def out_decrypt(alpha, v, x):
    a = hash(v + x)
    if G^a != alpha:
        return "Decryption failed"
    return x

$$\mathsf{Decrypt}(\mathit{key}, c) \;\equiv\;$$
$$\vartheta \,\alpha,v,x \gets \mathsf{in}(\mathit{key},c)\,\centerdot\,\mathsf{out}(\alpha,v,x)$$

In [ ]:

def Decrypt(key, alpha, v_prime, x_prime):
    alpha, v, x = in_decrypt(key, alpha, v_prime, x_prime)
    return out_decrypt(alpha, v, x)

In [ ]:
lmbda = 128
id = "sender@test.com"
x = 12345

s, beta = KeyGen(lmbda)
key = KeyExtract(id, s)
alpha, v_prime, x_prime = Encrypt(id, x, beta)
decrypted_x = Decrypt(key, alpha, v_prime, x_prime)
print("Original message:", x)
print("Decrypted message:", decrypted_x)